# Code Generation Demo

In [1]:
import transformers
import torch

In [2]:
## define run name
run_name = "MLPC-2048"
# run_name = "MLPC-2048-StarCoderBase7B"

# define model for tokenizer
model_name = "codellama/CodeLlama-7b-hf"
# model_name = "bigcode/starcoderbase-7b"

# dataset import folder
export_folder = "./dataset/" + run_name + "/"

# model save path
model_save_path = "./models/" + run_name + "/"

# model checkpoint path
model_checkpoint_path = "./checkpoints/" + run_name + "/"

In [3]:
## Test loading model and inference with that model

# load quantization config for 4bit quantization -> must be same as training
quantization_config = transformers.BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

# load model from model_save_path with quantization config
model = transformers.AutoModelForCausalLM.from_pretrained(model_save_path, quantization_config=quantization_config, low_cpu_mem_usage=True)

# optional: load model untrained
# model = transformers.AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, low_cpu_mem_usage=True)

# optional: load model from checkpoint
# model = transformers.AutoModelForCausalLM.from_pretrained("./output/bigRun/checkpoint-1000", quantization_config=quantization_config, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
# load tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# add pad token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

### Generate to `<END>`

In [5]:
# Prompt
# prompt = "<START>"
prompt ="<START> {\"nitrox.dlc.mirror.model.FieldModel\","

input_ids = tokenizer.encode(prompt, return_tensors='pt')

end_token_id = tokenizer.encode("<END>", add_special_tokens=False)[0]

In [6]:
# generate token to end token
output = model.generate(input_ids, eos_token_id=end_token_id, temperature=0.1, max_length=300)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)


/home/henrikwiegand/thesis/thesis-localllm-codetuning/FixCUDAShit/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:529 for open-end generation.
/home/henrikwiegand/thesis/thesis-localllm-codetuning/FixCUDAShit/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure t

In [7]:
# delete start token "<START>" from text
generated_text = generated_text.replace("<START>", "")
generated_text

' {"nitrox.dlc.mirror.model.FieldModel", "name": "VAR_name", "type": {"@class": "nitrox.dlc.mirror.model.AssertedContainableTypeModel", "typeName": "VAR_typeName", "domainType": "NON_DOMAIN", "assertions": [], "hasOptionalContainer": false, "hasCollectionContainer": false, "hasListContainer": false, "hasSetContainer": false, "hasStreamContainer": false, "containerTypeName": "VAR_containerTypeName", "containerAssertions": []}, "accessLevel": "PRIVATE", "declaredByTypeName": "VAR_declaredByTypeName", "modifiable": false, "publicReadable": true, "publicWriteable": false, "static": false}, {"@class": "nitrox.dlc.mirror.model.FieldModel", "name": "VAR_name", "type": {"@class": "nitrox.dlc.mirror.model.AssertedContainableTypeModel", "typeName": "VAR_typeName", "domainType": "NON_DOMAIN", "assertions": [], "hasOptionalContainer": false, "hasCollectionContainer": false, "hasListContainer": false, "hasSetContainer": false, "has'

In [8]:
# write generated text to generated_text.txt
with open("generated_text.json", "w") as file:
    file.write(generated_text)

### Test JSON Parsing

In [10]:
import json

# Try to parse the generated text as JSON
try:
    parsed_json = json.loads(generated_text)
    print("The generated text is valid JSON.")

    # Format the JSON
    formatted_json = json.dumps(parsed_json, indent=4)

    # save formatted json to file
    with open("generated_json.json", "w") as f:
        f.write(formatted_json)

except json.JSONDecodeError as e:
    print(f"The generated text is not valid JSON. Error: {e}")

The generated text is not valid JSON. Error: Expecting property name enclosed in double quotes: line 1 column 1111 (char 1110)


### Generate tokens while confidence

In [9]:
# Prompt
prompt = "<START>"
prompt ="<START> { \"@class\" : \"nitrox.dlc.mirror.model.AggregateRootModel\",\""

input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [10]:
import torch.nn.functional as F

# Encode the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Initialize the output as the input
output = input_ids

# Loop until the end token is generated
while True:
    # Predict the probabilities of the next token
    with torch.no_grad():
        outputs = model(output)
    predictions = outputs.logits[:, -1, :]
    probabilities = F.softmax(predictions, dim=-1)

    # Get the top 5 tokens and their probabilities
    top_probs, top_token_ids = torch.topk(probabilities, 5)

    # Print the top 5 tokens and their probabilities
    previous_text = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"\nPrevious text: {previous_text} add tokens:")
    for i in range(5):
        token = tokenizer.decode(top_token_ids[0][i])
        prob = top_probs[0][i].item()
        print(f"Token: {token}, Confidence: {prob}")

    # Get the token with the highest probability
    max_prob, max_token_id = torch.max(probabilities, dim=-1)

    # Check if the confidence is over 50%
    if max_prob.item() < 0.50:
        break

    # Append the token to the output
    output = torch.cat([output, max_token_id.unsqueeze(0)], dim=-1)

# Decode the output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)



Previous text: <START> { "@class" : "nitrox.dlc.mirror.model.AggregateRootModel"," add tokens:
Token: type, Confidence: 0.6565255522727966
Token: @, Confidence: 0.18374043703079224
Token: name, Confidence: 0.04573661461472511
Token: in, Confidence: 0.021103980019688606
Token: mir, Confidence: 0.017359659075737

Previous text: <START> { "@class" : "nitrox.dlc.mirror.model.AggregateRootModel","type add tokens:
Token: Name, Confidence: 0.9999967813491821
Token: Name, Confidence: 2.8573149393196218e-06
Token: NAME, Confidence: 2.496693980447162e-07
Token: name, Confidence: 6.214762748868452e-08
Token: Names, Confidence: 3.274949733622634e-08

Previous text: <START> { "@class" : "nitrox.dlc.mirror.model.AggregateRootModel","typeName add tokens:
Token: ", Confidence: 0.9946234226226807
Token: ":, Confidence: 0.0027936925180256367
Token: ":", Confidence: 0.002543685957789421
Token: ", Confidence: 9.919420335791074e-06
Token: <MID>, Confidence: 5.65191521673114e-06

Previous text: <START> { "

In [11]:
# write generated text to generated_text.txt
with open("generated_text.json", "w") as file:
    file.write(generated_text)